In [1]:
import os

os.environ["OMP_NUM_THREADS"] = "8"
import numpy as np

import hapod as hp

In [ ]:
hp.matrix_memory_footprint((3600000, 500)) / 2**30

In [ ]:
hp.svd_memory_footprint((3600000, 500)) / 2**30

In [6]:
np.random.seed(42)

X = hp.random_matrix(3600000, 500, 500)

In [7]:
U, s, VT = np.linalg.svd(X, full_matrices=False)